Updated notebook now that actually auto generates time stamps for the podcast so people can skip to their favourite section.

In [30]:
API_TOKEN = "" # will be importing my API key so I can use the features in LeMUR to

In [3]:
%pip install "assemblyai>=0.15" --quiet # install the AssemblyAI Python SDK which grants us access to LeMUR in just a few short lines of code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [31]:
import assemblyai as aai # import the package

In [32]:
aai.settings.api_key = f"{API_TOKEN}" # set the API token

In [48]:
transcriber = aai.Transcriber(config=aai.TranscriptionConfig(auto_chapters=True)) # create a transcriber object
transcript = transcriber.transcribe("/content/podcasts/Jesse_Hutabarat.mp3") # transcribe the podcast by passing through the file path located in the podcasts folder
if transcript.error:
    print(transcript.error)
else:
    print("Success!")


Success!


Successful transcription which means we now have access to a unique transcript ID

In [49]:
transcript.id

'455e56b9-b157-4eda-812d-3d580c8d3811'

In [50]:
TRANSCRIPT_ID = transcript.id

In [51]:
transcript_from_id = aai.Transcript.get_by_id(TRANSCRIPT_ID)

Now I will extend the functionality of this project to generate accurate and relevant time stamps for say a youtube video so people can skip to the parts they are most interested in.

In [52]:
# print the text
print(transcript.text, end='\n\n')

You have tuned in to the Evolve podcast, where we aim to solve worldwide strength and conditioning issues one episode at a time. So sit back, relax, grab your protein shake as we chat, chat with some of the greatest minds this industry has to offer. Welcome back to episode nine of the Evolve podcast. Today we have a very special guest, Jesse Hatabarat. Did I get that right? Yeah. Yep, you got that right. Sounds. It sounds about right. So Jesse is a powerlifter, he's a two time New South Wales state champion, and I believe he is number four ranked in the 93. Kilowatt had a very successful state champs in a couple of months ago and leading up to nationals in a month or two. What are we now? I think we are 36 days out. We're just a month out now. I'm pretty sure about six weeks or five and a half weeks now. Yeah, mate. Getting very excited. So we wanted to get Jesse on today, obviously talk about his journey, where he's come from. He's had a lot of success recently in the world of powerin

In [53]:
# now we print the video sections information
print(transcript.chapters)

[Chapter(summary=" episode nine of the Evolve podcast. Today we have a very special guest, Jesse Hatabarat. Jesse is a powerlifter, a two time New South Wales state champion. Also talks about the new federation that's coming in Australia, the APA.", headline='Evolve podcast features special guest Jesse Hatabarat from New South Wales', gist='Jesse Hatabarat', start=170, end=81346), Chapter(summary='Before we start, five rapid fire questions. You can take as long or as little as you need to answer these. If you can choose one for the rest of your life, what would it be? I always have a steak and cheese sandwich after training.', headline='Five rapid fire questions to help you improve after your workout', gist='5 Rapid Fire Questions', start=81378, end=157840), Chapter(summary="UFC fighters are getting a bit of a spin off in movies. There's a new one with Conor McGregor. If you could attend one sporting event in history, what would it be? UFC.", headline='Question number three. If you cou

In [54]:
for chapter in transcript.chapters:
    print(f"Start: {chapter.start}, End: {chapter.end}")
    print(f"Summary: {chapter.summary}")
    print(f"Headline: {chapter.headline}")
    print(f"Gist: {chapter.gist}")

Start: 170, End: 81346
Summary:  episode nine of the Evolve podcast. Today we have a very special guest, Jesse Hatabarat. Jesse is a powerlifter, a two time New South Wales state champion. Also talks about the new federation that's coming in Australia, the APA.
Headline: Evolve podcast features special guest Jesse Hatabarat from New South Wales
Gist: Jesse Hatabarat
Start: 81378, End: 157840
Summary: Before we start, five rapid fire questions. You can take as long or as little as you need to answer these. If you can choose one for the rest of your life, what would it be? I always have a steak and cheese sandwich after training.
Headline: Five rapid fire questions to help you improve after your workout
Gist: 5 Rapid Fire Questions
Start: 158450, End: 246458
Summary: UFC fighters are getting a bit of a spin off in movies. There's a new one with Conor McGregor. If you could attend one sporting event in history, what would it be? UFC.
Headline: Question number three. If you could attend on

In [55]:
def ms_to_hms(start):
    s, ms = divmod(start, 1000)
    m, s = divmod(s, 60)
    h, m = divmod(m, 60)
    return h, m, s

In [56]:
def create_timestamps(chapters):
  last_hour = ms_to_hms(chapters[-1].start)[0]
  time_format = "{m:02d}:{s:02d}" if last_hour == 0 else "{h:02d}:{m:02d}:{s:02d}"

  lines = []
  for idx, chapter in enumerate(chapters):
        # first YouTube timestamp must be at zero
        h, m, s = (0, 0, 0) if idx == 0 else ms_to_hms(chapter.start)
        lines.append(f"{time_format.format(h=h, m=m, s=s)} {chapter.headline}")

  return "\n".join(lines)

In [57]:
timestamp_lines = create_timestamps(transcript.chapters)
print(timestamp_lines)

00:00 Evolve podcast features special guest Jesse Hatabarat from New South Wales
01:21 Five rapid fire questions to help you improve after your workout
02:38 Question number three. If you could attend one sporting event in history, what would it be
04:06 So question number four. What's your life motto
04:32 What would you rather win nationals with an average total or come second
05:36 Jesse started powerlifting in 2015 when he was in year eleven
11:04 You met your first coach at your first power of Ingym
18:04 He's just so definitive. And I guess that's why I've been under him for so long now
18:20 You've been training powerlifting for 12 years now
24:57 Who stays in it the longest is the winner. We're already through the first month of the year this year
25:18 Looking into the future for Jesse, what are your plans
29:57 Callan Givens is someone that you want to beat at nationals
32:29 Sussey: I think the new federation is the perfect solution for Australian powerlifting
36:21 Matt: Pe

Definitely a solid start with some potential however it's all over the place. Let's use LLM's in particular LeMUR to improve the output.

In [58]:
prompt = f"""
ROLE:
You are a YouTube content professional. You are very competent and able to come up with catchy names for the different sections of video transcripts that are submitted to you.
CONTEXT:
This transcript is of a logistics meeting at GitLab
INSTRUCTION:
You are provided information about the sections of the transcript under TIMESTAMPS, where the format for each line is `<TIMESTAMP> <SECTION SUMMARY>`."
TIMESTAMPS:
{timestamp_lines}
FORMAT:
<TIMESTAMP> <CATCHY SECTION TITLE>
OUTPUT:
""".strip()

In [59]:
result = transcript.lemur.task(prompt)

# Extract the response text and print
output = result.response.strip()
print(output)

Here are catchy section titles for the transcript timestamps:

00:00 Jesse Hatabarat Joins Evolve Podcast  
01:21 Rapid Fire Q&A
02:38 Dream Sporting Event 
04:06 Life Motto Revealed
04:32 Win Nationals or Come Second?
05:36 Jesse's Powerlifting Origin Story  
11:04 Meeting His Longtime Coach
18:04 The Calculative Coach
18:20 12 Years of Powerlifting  
24:57 It's a Marathon, Not a Sprint
25:18 Jesse's Future Plans
29:57 Callan Givens Rivalry  
32:29 New Federation Praised
36:21 Social Media Push Before Nationals
38:32 Ideal New Federation Wishes
39:48 Final Words of Wisdom


In [60]:
import re

def filter_timestamps(text):
    lines = text.splitlines()
    timestamped_lines = [line for line in lines if re.match(r'\d+:\d+', line)]  # Use regex to filter lines starting with a timestamp
    filtered_text = '\n'.join(timestamped_lines)
    return filtered_text

filtered_output = filter_timestamps(output)

In [61]:
original = timestamp_lines.splitlines()
filtered = filtered_output.splitlines()

for o, f in zip(original, filtered):
    original_time = o.split(' ')[0]
    filtered_time = f.split(' ')[0]
    if not original_time == filtered_time:
        raise RuntimeError(f"Timestamp mismatch - original timestamp '{original_time}' does not match LLM timestamp '{filtered_time}'")

print(filtered_output)

00:00 Jesse Hatabarat Joins Evolve Podcast  
01:21 Rapid Fire Q&A
02:38 Dream Sporting Event 
04:06 Life Motto Revealed
04:32 Win Nationals or Come Second?
05:36 Jesse's Powerlifting Origin Story  
11:04 Meeting His Longtime Coach
18:04 The Calculative Coach
18:20 12 Years of Powerlifting  
24:57 It's a Marathon, Not a Sprint
25:18 Jesse's Future Plans
29:57 Callan Givens Rivalry  
32:29 New Federation Praised
36:21 Social Media Push Before Nationals
38:32 Ideal New Federation Wishes
39:48 Final Words of Wisdom
